# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open("train.csv").readlines())
val_doc = np.random.permutation(open("val.csv").readlines())
batch_size = 30  #experiment with the batch size

In [4]:
len(train_doc), len(val_doc)

(663, 100)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [10]:
# x = 30 # number of frames
# y = 120 # image width
# z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    resize = imresize(image,(y,z))
                    temp = resize/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10  # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

#write your model here

# Experiment -1 , Convolutional layers -2 , Dense layers -2 , Batch Normalization after every conv layer.
#                  Dropout after conv and Dense layers

model_1 = Sequential()
model_1.add(Conv3D(filters=32,kernel_size=(3,3,3), padding='same', input_shape = (30, 120, 120, 3)))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2,2,2)))
model_1.add(Dropout(0.25))


model_1.add(Conv3D(filters=16,kernel_size=(3,3,3), padding='same'))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2,2,2)))
model_1.add(Dropout(0.25))

# flatten the output from conv layers and feed in to Dense layers
model_1.add(Flatten())
model_1.add(Dense(units=256,activation='relu'))
model_1.add(Dropout(0.50))


# final softmax layer

model_1.add(Dense(units=5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [21]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation_3 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_4 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [22]:
train_path

'train'

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 40
Source path =  train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 1:32 - loss: 8.7216 - categorical_accuracy: 0.1625Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 83s 5s/step - loss: 11.9127 - categorical_accuracy: 0.2231 - val_loss: 13.2168 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-05-2414_16_02.324946/model-00001-11.89575-0.22323-13.21684-0.18000.h5
Epoch 2/10
17/17 [==============================] - 43s 3s/step - loss: 11.7275 - categorical_accuracy: 0.2685 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-05-2414_16_02.324946/model-00002-11.72750-0.26854-12.89448-0.20000.h5
Epoch 3/10
17/17 [==============================] - 41s 2s/step - loss: 11.8095 - categorical_accuracy: 0.2673 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-05-2414_16_02.324946/model-00003-11.81411-0.26703-12.08857-0.25000.h5
Epoch 4/10
17/17 [==============================] - 38s 2s/step - loss: 12.1287 - categorical_accuracy: 0.2446 - val_loss: 11.4356 - val_categorical_accuracy: 0.2833

#### loss: 10.7484 - categorical_accuracy: 0.3287 - val_loss: 11.5513 - val_categorical_accuracy: 0.2833
#### model-00010-10.74835-0.32872-11.55130-0.28333.h5

In [51]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    resize = imresize(image,(60,60))
                    temp = resize/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(60,60))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [52]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [53]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

#write your model here

# Experiment -2 , Convolutional layers -2 , Dense layers -2 , Batch Normalization after every conv layer.
#                  Dropout after conv and Dense layers , number of feature maps increased from 8 to 32 and 16 to 64,
#                 Dense Layer unit size changed from 256 to 512 , changed dimensions of image to (60,60,3)

model_2 = Sequential()
model_2.add(Conv3D(filters=8,kernel_size=(3,3,3), padding='same', input_shape = (30, 60, 60, 3)))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2,2,2)))
model_2.add(Dropout(0.25))


model_2.add(Conv3D(filters=16,kernel_size=(3,3,3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2,2,2)))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(filters=32,kernel_size=(3,3,3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2,2,2)))
model_2.add(Dropout(0.25))



# flatten the output from conv layers and feed in to Dense layers
model_2.add(Flatten())

model_2.add(Dense(units=100,activation='relu'))
model_2.add(Dropout(0.50))


# final softmax layer

model_2.add(Dense(units=5, activation='softmax'))


In [54]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 60, 60, 8)     656       
_________________________________________________________________
activation_19 (Activation)   (None, 30, 60, 60, 8)     0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 60, 60, 8)     32        
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 30, 30, 8)     0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 15, 30, 30, 8)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 15, 30, 30, 16)    3472      
_________________________________________________________________
activation_20 (Activation)   (None, 15, 30, 30, 16)    0         
__________

In [55]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [56]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 30
Source path =  train ; batch size = 30
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/23 [==>...........................] - ETA: 1:12 - loss: 5.6347 - categorical_accuracy: 0.2333Batch:  4 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 69s 3s/step - loss: 2.5032 - categorical_accuracy: 0.2526 - val_loss: 1.3675 - val_categorical_accuracy: 0.4700

Epoch 00001: saving model to model_init_2020-05-2414_16_02.324946/model-00001-2.53505-0.26244-1.36747-0.47000.h5
Epoch 2/10
23/23 [==============================] - 6s 246ms/step - loss: 1.5883 - categorical_accuracy: 0.2174 - val_loss: 1.5251 - val_categorical_accuracy: 0.4250

Epoch 00002: saving model to model_init_2020-05-2414_16_02.324946/model-00002-1.58833-0.21739-1.52511-0.42500.h5
Epoch 3/10
23/23 [==============================] - 8s 349ms/step - loss: 1.5737 - categorical_accuracy: 0.2754 - val_loss: 2.2311 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-05-2414_16_02.324946/model-00003-1.57369-0.27536-2.23110-0.25000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
23/23 [==============================] - 7s 300ms/step - loss: 1.4650 - categoric

In [ ]:
#### 

In [58]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height
batch_size = 20

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    resize = imresize(image,(60,60))
                    temp = resize/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(60,60))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [59]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10  # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [64]:

# Experiment -3 , Changing batch size to 20 and removing BN , addition of conv and 
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 30 # number of frames
y = 60 # image width
z = 60 # image height

model_3 = Sequential()
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(5, activation='softmax'))

In [65]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_34 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_34 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_35 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_35 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_28 (MaxPooling (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_36 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [66]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [67]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [68]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = 20
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 4s - loss: 1.6531 - categorical_accuracy: 0.1859Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 74s 2s/step - loss: 1.6662 - categorical_accuracy: 0.1840 - val_loss: 1.5121 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-05-2414_16_02.324946/model-00001-1.64903-0.18854-1.51206-0.23000.h5
Epoch 2/10
34/34 [==============================] - 9s 276ms/step - loss: 1.6048 - categorical_accuracy: 0.1765 - val_loss: 1.6112 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-2414_16_02.324946/model-00002-1.60482-0.17647-1.61125-0.21000.h5
Epoch 3/10
34/34 [==============================] - 14s 397ms/step - loss: 1.6133 - categorical_accuracy: 0.1667 - val_loss: 1.6113 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-05-2414_16_02.324946/model-00003-1.61334-0.16667-1.61134-0.21000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
34/34 [==============================] - 11s 330ms/step - loss: 1.6136 - categor

In [73]:
# model_3 - change the batch size to 20

train_generator = generator(train_path, train_doc, 20)
val_generator = generator(val_path, val_doc, 20)





In [74]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = 20
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 4s - loss: 1.6096 - categorical_accuracy: 0.2078Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 71s 2s/step - loss: 1.6098 - categorical_accuracy: 0.2097 - val_loss: 1.6101 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-05-2414_16_02.324946/model-00001-1.60974-0.20664-1.61012-0.21000.h5
Epoch 2/10
34/34 [==============================] - 12s 356ms/step - loss: 1.6056 - categorical_accuracy: 0.3137 - val_loss: 1.6100 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-2414_16_02.324946/model-00002-1.60558-0.31373-1.60995-0.21000.h5
Epoch 3/10
34/34 [==============================] - 13s 368ms/step - loss: 1.6087 - categorical_accuracy: 0.1961 - val_loss: 1.6089 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-05-2414_16_02.324946/model-00003-1.60869-0.19608-1.60889-0.21000.h5
Epoch 4/10
34/34 [==============================] - 12s 366ms/step - loss: 1.5937 - categorical_accuracy: 0.2059 - val_loss: 1.5878 - val_categorical_accuracy: 0.1800

Epo

In [98]:
x = 15 # number of frames
y = 60 # image width
z = 60 # image height
batch_size = 20

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    resize = imresize(image,(90,90))
                    temp = resize/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(90,90))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [15]:
# Experiment -4 , Changing batch size to 20 and removing BN , addition of conv and 
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 15 # number of frames
y = 60 # image width
z = 60 # image height

model_4 = Sequential()
model_4.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_4.add(Dropout(0.25))

model_4.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_4.add(Dropout(0.25))

model_4.add(Flatten())
model_4.add(Dense(512, activation='relu'))
model_4.add(Dropout(0.5))
model_4.add(Dense(5, activation='softmax'))

In [16]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 15, 60, 60, 64)    5248      
_________________________________________________________________
activation_4 (Activation)    (None, 15, 60, 60, 64)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 60, 60, 64)    110656    
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 64)    0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 5, 20, 20, 64)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 20, 20, 64)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 5, 20, 20, 32)     55328     
__________

In [21]:
# changing batch size to 30 

x = 15 # number of frames
y = 90 # image width
z = 90 # image height
batch_size = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [69]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [24]:
model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 30
Source path =  train ; batch size = 30
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/23 [==>...........................] - ETA: 1:40 - loss: 1.8006 - categorical_accuracy: 0.1889Batch:  4 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 107s 5s/step - loss: 1.6302 - categorical_accuracy: 0.2120 - val_loss: 1.5711 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2020-05-2417_26_11.161846/model-00001-1.63165-0.22021-1.57112-0.29000.h5
Epoch 2/10
23/23 [==============================] - 4s 183ms/step - loss: 1.5756 - categorical_accuracy: 0.2899 - val_loss: 1.4371 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-05-2417_26_11.161846/model-00002-1.57563-0.28986-1.43710-0.25000.h5
Epoch 3/10
23/23 [==============================] - 4s 184ms/step - loss: 1.5141 - categorical_accuracy: 0.3043 - val_loss: 1.6123 - val_categorical_accuracy: 0.2250

Epoch 00003: saving model to model_init_2020-05-2417_26_11.161846/model-00003-1.51413-0.30435-1.61228-0.22500.h5
Epoch 4/10
23/23 [==============================] - 4s 183ms/step - loss: 1.5860 - categorical_accuracy: 0.2609 - val_loss: 1.6625 - val_categorical_accuracy: 0.1250

Epoch

In [34]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 20 # number of frames
y = 60 # image width
z = 60 # image height

model_5 = Sequential()
model_5.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))

model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))

model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))


model_5.add(Flatten())
model_5.add(Dense(512, activation='relu'))
model_5.add(Dropout(0.5))

model_5.add(Dense(512, activation='relu'))
model_5.add(Dropout(0.5))


model_5.add(Dense(5, activation='softmax'))

In [35]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 20, 60, 60, 64)    5248      
_________________________________________________________________
activation_15 (Activation)   (None, 20, 60, 60, 64)    0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 20, 60, 60, 64)    110656    
_________________________________________________________________
activation_16 (Activation)   (None, 20, 60, 60, 64)    0         
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 7, 20, 20, 64)     0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 7, 20, 20, 64)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 7, 20, 20, 64)     110656    
__________

In [36]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [37]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [38]:
model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 30
Source path =  train ; batch size = 30
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/23 [==>...........................] - ETA: 1:56 - loss: 1.6196 - categorical_accuracy: 0.2000Batch:  4 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 118s 5s/step - loss: 1.6102 - categorical_accuracy: 0.2230 - val_loss: 1.6148 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-05-2417_26_11.161846/model-00001-1.61315-0.21870-1.61480-0.25000.h5
Epoch 2/10
23/23 [==============================] - 6s 241ms/step - loss: 1.6183 - categorical_accuracy: 0.3333 - val_loss: 1.5900 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-05-2417_26_11.161846/model-00002-1.61826-0.33333-1.58999-0.20000.h5
Epoch 3/10
23/23 [==============================] - 6s 240ms/step - loss: 1.6180 - categorical_accuracy: 0.1739 - val_loss: 1.6021 - val_categorical_accuracy: 0.2250

Epoch 00003: saving model to model_init_2020-05-2417_26_11.161846/model-00003-1.61801-0.17391-1.60206-0.22500.h5
Epoch 4/10
23/23 [==============================] - 6s 240ms/step - loss: 1.6215 - categorical_accuracy: 0.2319 - val_loss: 1.5674 - val_categorical_accuracy: 0.2500

Epoch

In [39]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 25 # number of frames
y = 90 # image width
z = 90 # image height

model_5 = Sequential()
model_5.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))

model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))

model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))


model_5.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_5.add(Dropout(0.1))


model_5.add(Flatten())
model_5.add(Dense(1000, activation='relu'))
model_5.add(Dropout(0.5))

model_5.add(Dense(512, activation='relu'))
model_5.add(Dropout(0.5))


model_5.add(Dense(5, activation='softmax'))

In [40]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 25, 90, 90, 64)    5248      
_________________________________________________________________
activation_19 (Activation)   (None, 25, 90, 90, 64)    0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 25, 90, 90, 64)    110656    
_________________________________________________________________
activation_20 (Activation)   (None, 25, 90, 90, 64)    0         
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 9, 30, 30, 64)     0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 9, 30, 30, 64)     0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 9, 30, 30, 64)     110656    
__________

In [41]:
batch_size = 30

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [44]:
model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=50, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/23 [==>...........................] - ETA: 2:24 - loss: 1.5970 - categorical_accuracy: 0.2444Batch:  23 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 44s 2s/step - loss: 1.6223 - categorical_accuracy: 0.2981 - val_loss: 1.6164 - val_categorical_accuracy: 0.1750

Epoch 00001: saving model to model_init_2020-05-2417_26_11.161846/model-00001-1.60540-0.25424-1.61644-0.17500.h5
Epoch 2/50
23/23 [==============================] - 15s 649ms/step - loss: 1.6197 - categorical_accuracy: 0.1739 - val_loss: 1.6026 - val_categorical_accuracy: 0.2250

Epoch 00002: saving model to model_init_2020-05-2417_26_11.161846/model-00002-1.61968-0.17391-1.60262-0.22500.h5
Epoch 3/50
23/23 [==============================] - 15s 648ms/step - loss: 1.6084 - categorical_accuracy: 0.1739 - val_loss: 1.6317 - val_categorical_accuracy: 0.0750

Epoch 00003: saving model to model_init_2020-05-2417_26_11.161846/model-00003-1.60839-0.17391-1.63166-0.07500.h5
Epoch 4/50
23/23 [==============================] - 15s 652ms/step - loss: 1.6112 - categorical_accuracy: 0.2174 - val_loss: 1.5609 - val_categorical_accuracy: 0.4750

Epo

23/23 [==============================] - 15s 652ms/step - loss: 1.4072 - categorical_accuracy: 0.3188 - val_loss: 1.3332 - val_categorical_accuracy: 0.2750

Epoch 00029: saving model to model_init_2020-05-2417_26_11.161846/model-00029-1.40717-0.31884-1.33320-0.27500.h5
Epoch 30/50
23/23 [==============================] - 15s 651ms/step - loss: 1.4061 - categorical_accuracy: 0.3913 - val_loss: 1.2851 - val_categorical_accuracy: 0.5250

Epoch 00030: saving model to model_init_2020-05-2417_26_11.161846/model-00030-1.40608-0.39130-1.28509-0.52500.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 31/50
23/23 [==============================] - 15s 653ms/step - loss: 1.4095 - categorical_accuracy: 0.3333 - val_loss: 1.3447 - val_categorical_accuracy: 0.3750

Epoch 00031: saving model to model_init_2020-05-2417_26_11.161846/model-00031-1.40953-0.33333-1.34470-0.37500.h5
Epoch 32/50
23/23 [==============================] - 15s 652ms/step - loss: 1.3102 - 

In [54]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 25 # number of frames
y = 120 # image width
z = 120 # image height

model_6 = Sequential()
model_6.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_6.add(Activation('relu'))
model_6.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_6.add(Activation('relu'))
model_6.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_6.add(Dropout(0.2))

model_6.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_6.add(Activation('relu'))
model_6.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_6.add(Activation('relu'))
model_6.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_6.add(Dropout(0.2))




model_6.add(Flatten())
model_6.add(Dense(512, activation='relu'))
model_6.add(Dropout(0.5))

model_6.add(Dense(256, activation='relu'))
model_6.add(Dropout(0.5))


model_6.add(Dense(5, activation='softmax'))
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_6.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_36 (Conv3D)           (None, 25, 120, 120, 64)  5248      
_________________________________________________________________
activation_36 (Activation)   (None, 25, 120, 120, 64)  0         
_________________________________________________________________
conv3d_37 (Conv3D)           (None, 25, 120, 120, 32)  55328     
_________________________________________________________________
activation_37 (Activation)   (None, 25, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_24 (MaxPooling (None, 9, 40, 40, 32)     0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 9, 40, 40, 32)     0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 9, 40, 40, 64)     55360     
__________

In [55]:
batch_size = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [56]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [57]:
model_6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  Epoch 1/20train ; batch size = 
20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/34 [===================>..........] - ETA: 37s - loss: 1.6275 - categorical_accuracy: 0.1630Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 118s 3s/step - loss: 1.6230 - categorical_accuracy: 0.1773 - val_loss: 1.6085 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-05-2417_26_11.161846/model-00001-1.62342-0.17345-1.60853-0.21000.h5
Epoch 2/20
34/34 [==============================] - 29s 846ms/step - loss: 1.6088 - categorical_accuracy: 0.1961 - val_loss: 1.6080 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-05-2417_26_11.161846/model-00002-1.60884-0.19608-1.60796-0.23000.h5
Epoch 3/20
34/34 [==============================] - 29s 847ms/step - loss: 1.6178 - categorical_accuracy: 0.2255 - val_loss: 1.6079 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-05-2417_26_11.161846/model-00003-1.61778-0.22549-1.60792-0.23000.h5
Epoch 4/20
34/34 [==============================] - 29s 847ms/step - loss: 1.6087 - categorical_accuracy: 0.2157 - val_loss: 1.6079 - val_categorical_accuracy: 0.2300

Ep

KeyboardInterrupt: 

In [60]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 20 # number of frames
y = 72 # image width
z = 72 # image height

model_7 = Sequential()
model_7.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_7.add(Activation('relu'))
model_7.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_7.add(Activation('relu'))
model_7.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
model_7.add(Dropout(0.2))

model_7.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_7.add(Activation('relu'))
model_7.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_7.add(Activation('relu'))
model_7.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
model_7.add(Dropout(0.2))




model_7.add(Flatten())
model_7.add(Dense(512, activation='relu'))
model_7.add(Dropout(0.5))

model_7.add(Dense(256, activation='relu'))
model_7.add(Dropout(0.5))


model_7.add(Dense(5, activation='softmax'))
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_48 (Conv3D)           (None, 20, 72, 72, 64)    5248      
_________________________________________________________________
activation_48 (Activation)   (None, 20, 72, 72, 64)    0         
_________________________________________________________________
conv3d_49 (Conv3D)           (None, 20, 72, 72, 32)    55328     
_________________________________________________________________
activation_49 (Activation)   (None, 20, 72, 72, 32)    0         
_________________________________________________________________
max_pooling3d_30 (MaxPooling (None, 10, 36, 36, 32)    0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 10, 36, 36, 32)    0         
_________________________________________________________________
conv3d_50 (Conv3D)           (None, 10, 36, 36, 64)    55360     
__________

In [61]:
batch_size = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [62]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [63]:
model_7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 3s - loss: 1.6820 - categorical_accuracy: 0.1953Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 56s 2s/step - loss: 1.6774 - categorical_accuracy: 0.2009 - val_loss: 1.6085 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-05-2417_26_11.161846/model-00001-1.67939-0.19759-1.60851-0.23000.h5
Epoch 2/20
34/34 [==============================] - 11s 316ms/step - loss: 1.6103 - categorical_accuracy: 0.1471 - val_loss: 1.6061 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-2417_26_11.161846/model-00002-1.61027-0.14706-1.60615-0.21000.h5
Epoch 3/20
34/34 [==============================] - 11s 317ms/step - loss: 1.6057 - categorical_accuracy: 0.2549 - val_loss: 1.6068 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-05-2417_26_11.161846/model-00003-1.60567-0.25490-1.60681-0.21000.h5
Epoch 4/20
34/34 [==============================] - 11s 317ms/step - loss: 1.6135 - categorical_accuracy: 0.1863 - val_loss: 1.6084 - val_categorical_accuracy: 0.2100

Epo

KeyboardInterrupt: 

In [42]:
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 15 # number of frames
y = 100 # image width
z = 100 # image height
batch_size = 20

model_8 = Sequential()
model_8.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_8.add(Activation('relu'))
model_8.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_8.add(Activation('relu'))
model_8.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
model_8.add(Dropout(0.25))

model_8.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_8.add(Activation('relu'))
model_8.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_8.add(Activation('relu'))
model_8.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
model_8.add(Dropout(0.25))



model_8.add(Flatten())
model_8.add(Dense(128, activation='relu'))
model_8.add(Dropout(0.5))

model_8.add(Dense(64, activation='relu'))
model_8.add(Dropout(0.5))


model_8.add(Dense(5, activation='softmax'))
model_8.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_8.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_39 (Conv3D)           (None, 15, 100, 100, 64)  5248      
_________________________________________________________________
activation_39 (Activation)   (None, 15, 100, 100, 64)  0         
_________________________________________________________________
conv3d_40 (Conv3D)           (None, 15, 100, 100, 32)  55328     
_________________________________________________________________
activation_40 (Activation)   (None, 15, 100, 100, 32)  0         
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 8, 50, 50, 32)     0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 8, 50, 50, 32)     0         
_________________________________________________________________
conv3d_41 (Conv3D)           (None, 8, 50, 50, 64)     55360     
__________

In [43]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [44]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
model_8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 20
Source path =  train ; batch size = 20
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/34 [===================>..........] - ETA: 24s - loss: 1.8092 - categorical_accuracy: 0.1826Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 74s 2s/step - loss: 1.7392 - categorical_accuracy: 0.2001 - val_loss: 1.6082 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-05-2418_47_46.240132/model-00001-1.74730-0.18854-1.60820-0.18000.h5
Epoch 2/20
34/34 [==============================] - 15s 446ms/step - loss: 1.6316 - categorical_accuracy: 0.1569 - val_loss: 1.6086 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-2418_47_46.240132/model-00002-1.63158-0.15686-1.60861-0.21000.h5
Epoch 3/20
34/34 [==============================] - 15s 447ms/step - loss: 1.6501 - categorical_accuracy: 0.1667 - val_loss: 1.6092 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-05-2418_47_46.240132/model-00003-1.65011-0.16667-1.60921-0.22000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
34/34 [==============================] - 15s 445ms/step - loss: 1.6100 - categorical_accuracy: 0.

In [20]:
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 25 # number of frames
y = 60 # image width
z = 60 # image height
batch_size = 10

model_9 = Sequential()
model_9.add(Conv3D(8, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_9.add(Activation('relu'))
model_9.add(Conv3D(16, kernel_size=(3, 3, 3), padding='same'))
model_9.add(Activation('relu'))
model_9.add(MaxPooling3D(pool_size=(2, 2, 2)))
model_9.add(Dropout(0.25))

model_9.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
model_9.add(Activation('relu'))
model_9.add(MaxPooling3D(pool_size=(2,2,2)))
model_9.add(Dropout(0.25))


model_9.add(Flatten())

model_9.add(Dense(128, activation='relu'))
model_9.add(Dropout(0.5))

model_9.add(Dense(84, activation='relu'))
model_9.add(Dropout(0.5))



# model_9.add(Conv3D(8, kernel_size=(3, 3,3), activation='relu',padding='same',input_shape=(x,y,z,3)))
# #model_9.add(MaxPooling3D(pool_size=(2, 2,2),strides=1, padding='same'))
# model_9.add(Conv3D(16, kernel_size=(5,5,5), activation='relu', padding='same'))
# model_9.add(MaxPooling3D(pool_size=(2, 2,2),strides=1, padding='same'))
# model_9.add(Dropout(0.25))



# model_9.add(Flatten())
# model_9.add(Dense(120, activation='relu'))
# model_9.add(Dropout(0.5))


model_9.add(Dense(5, activation='softmax'))
model_9.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_9.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_20 (Conv3D)           (None, 25, 60, 60, 8)     656       
_________________________________________________________________
activation_20 (Activation)   (None, 25, 60, 60, 8)     0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 25, 60, 60, 16)    3472      
_________________________________________________________________
activation_21 (Activation)   (None, 25, 60, 60, 16)    0         
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 12, 30, 30, 16)    0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 12, 30, 30, 32)    13856     
_________________________________________________________________
activation_22 (Activation)   (None, 12, 30, 30, 32)    0         
__________

In [21]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [24]:
model_9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=50, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  train ; batch size = 10
Epoch 1/50val ; batch size = 10



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 0s - loss: 1.6116 - categorical_accuracy: 0.1955

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 58s 866ms/step - loss: 1.6108 - categorical_accuracy: 0.1975 - val_loss: 1.5990 - val_categorical_accuracy: 0.4100

Epoch 00001: val_loss improved from inf to 1.59895, saving model to model_init_2020-05-2513_02_07.143992/model-00001-1.61136-0.19608-1.59895-0.41000.h5
Epoch 2/50
67/67 [==============================] - 19s 280ms/step - loss: 1.5854 - categorical_accuracy: 0.2239 - val_loss: 1.4992 - val_categorical_accuracy: 0.3600

Epoch 00002: val_loss improved from 1.59895 to 1.49922, saving model to model_init_2020-05-2513_02_07.143992/model-00002-1.58539-0.22388-1.49922-0.36000.h5
Epoch 3/50
67/67 [==============================] - 20s 303ms/step - loss: 1.5691 - categorical_accuracy: 0.3632 - val_loss: 1.5733 - val_categorical_accuracy: 0.2900

Epoch 00003: val_loss did not improve from 1.49922
Epoch 4/50
67/67 [==============================] - 19s 281ms/step - loss: 1.5937 - categorical_accuracy: 0.2836 - val_loss: 1.5228 - val_categorica

In [30]:
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

x = 25 # number of frames
y = 60 # image width
z = 60 # image height
batch_size = 10

model_10 = Sequential()
model_10.add(Conv3D(8, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_10.add(Activation('relu'))
model_10.add(Conv3D(16, kernel_size=(3, 3, 3), padding='same'))
model_10.add(Activation('relu'))
model_10.add(MaxPooling3D(pool_size=(2, 2, 2)))
model_10.add(Dropout(0.10))

model_10.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
model_10.add(Activation('relu'))
model_10.add(MaxPooling3D(pool_size=(2,2,2)))
model_10.add(Dropout(0.10))


model_10.add(Flatten())

model_10.add(Dense(128, activation='relu'))
model_10.add(Dropout(0.25))

model_10.add(Dense(84, activation='relu'))
model_10.add(Dropout(0.25))




model_10.add(Dense(5, activation='softmax'))
model_10.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_10.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_26 (Conv3D)           (None, 25, 60, 60, 8)     656       
_________________________________________________________________
activation_26 (Activation)   (None, 25, 60, 60, 8)     0         
_________________________________________________________________
conv3d_27 (Conv3D)           (None, 25, 60, 60, 16)    3472      
_________________________________________________________________
activation_27 (Activation)   (None, 25, 60, 60, 16)    0         
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 12, 30, 30, 16)    0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 12, 30, 30, 16)    0         
_________________________________________________________________
conv3d_28 (Conv3D)           (None, 12, 30, 30, 32)    13856     
__________

In [31]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [33]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [34]:
model_10.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=50, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 0s - loss: 1.5969 - categorical_accuracy: 0.2288

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 58s 865ms/step - loss: 1.6000 - categorical_accuracy: 0.2254 - val_loss: 1.4374 - val_categorical_accuracy: 0.3400

Epoch 00001: val_loss improved from inf to 1.43736, saving model to model_init_2020-05-2513_02_07.143992/model-00001-1.59785-0.22775-1.43736-0.34000.h5
Epoch 2/50
67/67 [==============================] - 18s 266ms/step - loss: 1.4542 - categorical_accuracy: 0.3532 - val_loss: 1.4505 - val_categorical_accuracy: 0.4500

Epoch 00002: val_loss did not improve from 1.43736
Epoch 3/50
67/67 [==============================] - 20s 298ms/step - loss: 1.4898 - categorical_accuracy: 0.3483 - val_loss: 1.3363 - val_categorical_accuracy: 0.4200

Epoch 00003: val_loss improved from 1.43736 to 1.33628, saving model to model_init_2020-05-2513_02_07.143992/model-00003-1.48978-0.34826-1.33628-0.42000.h5
Epoch 4/50
67/67 [==============================] - 19s 288ms/step - loss: 1.4790 - categorical_accuracy: 0.3582 - val_loss: 1.2699 - val_categorica

#### Model is overfitting. Train accuracy = 95% and validation accuracy = 72% till here

## Adding dropouts and L2 Regularization

In [42]:
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout
from keras.regularizers import l2

x = 30 # number of frames
y = 60 # image width
z = 60 # image height
batch_size = 10

model_11 = Sequential()
model_11.add(Conv3D(8, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
model_11.add(Activation('relu'))
model_11.add(Conv3D(16, kernel_size=(3, 3, 3), padding='same'))
model_11.add(Activation('relu'))
model_11.add(MaxPooling3D(pool_size=(2, 2, 2)))
model_11.add(Dropout(0.25))


model_11.add(Flatten())

model_11.add(Dense(48, activation='relu',kernel_regularizer=l2(0.001)))
model_11.add(Dropout(0.5))


model_11.add(Dense(5, activation='softmax'))
model_11.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_11.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 60, 60, 8)     656       
_________________________________________________________________
activation_17 (Activation)   (None, 30, 60, 60, 8)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 30, 60, 60, 16)    3472      
_________________________________________________________________
activation_18 (Activation)   (None, 30, 60, 60, 16)    0         
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 15, 30, 30, 16)    0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 15, 30, 30, 16)    0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 216000)            0         
__________

In [43]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [44]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [45]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [46]:
model_11.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  trainEpoch 1/30
 ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 2.0493 - categorical_accuracy: 0.2288

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 70s 1s/step - loss: 2.0316 - categorical_accuracy: 0.2402 - val_loss: 1.7201 - val_categorical_accuracy: 0.3100

Epoch 00001: val_loss improved from inf to 1.72013, saving model to model_init_2020-05-2514_50_20.061003/model-00001-2.04387-0.23228-1.72013-0.31000.h5
Epoch 2/30
67/67 [==============================] - 22s 332ms/step - loss: 1.6490 - categorical_accuracy: 0.3532 - val_loss: 1.6093 - val_categorical_accuracy: 0.4100

Epoch 00002: val_loss improved from 1.72013 to 1.60929, saving model to model_init_2020-05-2514_50_20.061003/model-00002-1.64895-0.35323-1.60929-0.41000.h5
Epoch 3/30
67/67 [==============================] - 23s 340ms/step - loss: 1.6568 - categorical_accuracy: 0.4030 - val_loss: 1.5745 - val_categorical_accuracy: 0.4900

Epoch 00003: val_loss improved from 1.60929 to 1.57450, saving model to model_init_2020-05-2514_50_20.061003/model-00003-1.65677-0.40299-1.57450-0.49000.h5
Epoch 4/30
67/67 [============================

67/67 [==============================] - 22s 335ms/step - loss: 0.6320 - categorical_accuracy: 0.8458 - val_loss: 1.1596 - val_categorical_accuracy: 0.6700

Epoch 00030: val_loss did not improve from 1.14708


Performance improvement seen after regularization

## CNN + RNN

In [11]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, Activation, Reshape, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

##### 2D-CNN and RNN model
# input size (batch_size=20, x=15, y=120, z=120, channel=3)
x=30
y=120
z=120
batch_size=10
# input layer
input_tensor = Input(shape=(x, y, z, 3))

# Get pre-trained model: vgg16 
vgg_layers = VGG16(weights='imagenet', include_top=False)
    # freeze the layers in base model
for layer in vgg_layers.layers:
    layer.trainable = False
    # create a VGG model selecting all layers   
vgg_model = Model(inputs=vgg_layers.input, outputs=vgg_layers.output)


# Adding Time Distributed wrapper on top of resnet model and passing the input tensor
time_distributed_layer= TimeDistributed(vgg_model)(input_tensor)

# Average pooling layer
avg_pool_layer= TimeDistributed(GlobalAveragePooling2D())(time_distributed_layer)

# Flatten before connecting to GRU
flatten_layer = TimeDistributed(Flatten())(avg_pool_layer)
flatten_dropped_out=Dropout(0.5)(flatten_layer)

# reshape the output of time distributed layer to be fed into LSTM or GRU
tensor_size=np.prod(time_distributed_layer.get_shape().as_list()[2:]) 
reshape_layer = Reshape(target_shape=(x,tensor_size))(time_distributed_layer)
         
# GRU layer
gru_out = GRU(256, return_sequences=False, dropout=0.5)(flatten_dropped_out)

# Fully connected Dense Layer
fc_out = Dense(256, activation="relu")(gru_out)
fc_dropped_out=Dropout(0.5)(fc_out)
bn_layer=BatchNormalization()(fc_dropped_out)

# Output layer
output = Dense(5, activation='softmax')(gru_out)

# final Model
model2 = Model(inputs=input_tensor, outputs=output)

In [12]:
optimiser = 'adam'
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 30, 120, 120, 3)   0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 512)           0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 512)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               590592    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 1285      
Total para

In [13]:
# if not experiment use experiment=0
experiment=0

if experiment!=0 and batch_size>experiment:
    batch_size=experiment

curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'

if experiment!=0:
    num_train_sequences = len(train_doc[:experiment])
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc[:experiment])
    print('# validation sequences =', num_val_sequences)
else:
    num_train_sequences = len(train_doc)
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [14]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [15]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2, cooldown=1)
callbacks_list = [checkpoint, LR]

In [16]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [17]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.6756 - categorical_accuracy: 0.2769Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 92s 1s/step - loss: 1.6722 - categorical_accuracy: 0.2781 - val_loss: 1.4254 - val_categorical_accuracy: 0.4300

Epoch 00001: val_loss improved from inf to 1.42540, saving model to model_init_2020-05-2517_43_24.798127/model-00001-1.67369-0.27753-1.42540-0.43000.h5
Epoch 2/20
67/67 [==============================] - 31s 462ms/step - loss: 1.5249 - categorical_accuracy: 0.3383 - val_loss: 1.2701 - val_categorical_accuracy: 0.4800

Epoch 00002: val_loss improved from 1.42540 to 1.27005, saving model to model_init_2020-05-2517_43_24.798127/model-00002-1.52490-0.33831-1.27005-0.48000.h5
Epoch 3/20
67/67 [==============================] - 31s 456ms/step - loss: 1.5172 - categorical_accuracy: 0.3333 - val_loss: 1.3501 - val_categorical_accuracy: 0.4100

Epoch 00003: val_loss did not improve from 1.27005
Epoch 4/20
67/67 [==============================] - 31s 460ms/step - loss: 1.4250 - categorical_accuracy: 0.4179 - val_loss: 1.2489 - val_categorical_a

# Experiment No.2

In [24]:
x = 30
y = 120
z = 120

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)]  #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [25]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [26]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, Activation, Reshape, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

##### 2D-CNN and RNN model
# input size (batch_size=20, x=15, y=120, z=120, channel=3)

# input layer
input_tensor = Input(shape=(x, y, z, 3))

# Get pre-trained model: vgg16 
vgg_layers = VGG16(weights='imagenet', include_top=False)
    # freeze the layers in base model
for layer in vgg_layers.layers:
    layer.trainable = False
    # create a VGG model selecting all layers   
vgg_model = Model(inputs=vgg_layers.input, outputs=vgg_layers.output)


# Adding Time Distributed wrapper on top of resnet model and passing the input tensor
time_distributed_layer= TimeDistributed(vgg_model)(input_tensor)

# Average pooling layer
avg_pool_layer= TimeDistributed(GlobalAveragePooling2D())(time_distributed_layer)

# Flatten before connecting to GRU
flatten_layer = TimeDistributed(Flatten())(avg_pool_layer)
flatten_dropped_out=Dropout(0.5)(flatten_layer)

# reshape the output of time distributed layer to be fed into LSTM or GRU
tensor_size=np.prod(time_distributed_layer.get_shape().as_list()[2:]) 
reshape_layer = Reshape(target_shape=(x,tensor_size))(time_distributed_layer)
         
# GRU layer
gru_out = GRU(256, return_sequences=False, dropout=0.5)(flatten_dropped_out)

# Fully connected Dense Layer
fc_out = Dense(256, activation="relu")(gru_out)
fc_dropped_out=Dropout(0.5)(fc_out)
bn_layer=BatchNormalization()(fc_dropped_out)

# Output layer
output = Dense(5, activation='softmax')(gru_out)

# final Model
model2 = Model(inputs=input_tensor, outputs=output)

In [27]:
optimiser = 'adam'
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 30, 120, 120, 3)   0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 30, 3, 3, 512)     14714688  
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 512)           0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 512)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
gru_5 (GRU)                  (None, 256)               590592    
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 1285      
Total para

In [28]:
# if not experiment use experiment=0
experiment=0

if experiment!=0 and batch_size>experiment:
    batch_size=experiment

curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'

if experiment!=0:
    num_train_sequences = len(train_doc[:experiment])
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc[:experiment])
    print('# validation sequences =', num_val_sequences)
else:
    num_train_sequences = len(train_doc)
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [29]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [30]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2, cooldown=1)
callbacks_list = [checkpoint, LR]

In [31]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [32]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  train ; batch size = 10
Epoch 1/20
 val ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.5862 - categorical_accuracy: 0.3277Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 88s 1s/step - loss: 1.5724 - categorical_accuracy: 0.3342 - val_loss: 1.1594 - val_categorical_accuracy: 0.5700

Epoch 00001: val_loss improved from inf to 1.15944, saving model to model_init_2020-05-2518_05_33.697998/model-00001-1.58154-0.32730-1.15944-0.57000.h5
Epoch 2/20
67/67 [==============================] - 31s 456ms/step - loss: 1.2952 - categorical_accuracy: 0.4826 - val_loss: 1.1561 - val_categorical_accuracy: 0.5400

Epoch 00002: val_loss improved from 1.15944 to 1.15608, saving model to model_init_2020-05-2518_05_33.697998/model-00002-1.29516-0.48259-1.15608-0.54000.h5
Epoch 3/20
67/67 [==============================] - 30s 441ms/step - loss: 1.3295 - categorical_accuracy: 0.4428 - val_loss: 1.0976 - val_categorical_accuracy: 0.5400

Epoch 00003: val_loss improved from 1.15608 to 1.09757, saving model to model_init_2020-05-2518_05_33.697998/model-00003-1.32949-0.44279-1.09757-0.54000.h5
Epoch 4/20
67/67 [============================